In [1]:
import os
os.chdir(os.path.join(os.getcwd(),'..'))

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import random
from collections import OrderedDict
from time import time

import numpy as np
import pandas as pd
from cameo import models
from retrying import retry

from analysis import BaseFVA

In [4]:
bigg_models = models.bigg

In [5]:
models = OrderedDict({
    'e_coli_core': BaseFVA.create_for(bigg_models.e_coli_core),
    'iAB_RBC_283': BaseFVA.create_for(bigg_models.iAB_RBC_283),
    'iRC1080': BaseFVA.create_for(bigg_models.iRC1080),
    'RECON1': BaseFVA.create_for(bigg_models.RECON1),
    'RECON2': BaseFVA.create_for()
})

In [6]:
@retry
def sample_analysis_time(model, size):
    input_data = { m.id: random.uniform(-10, 10) for m in random.sample(model.metabolites, size)}
    t = time()
    model.analyze(input_data)
    return time() - t

In [ ]:
from IPython.display import clear_output

In [ ]:
measurements = dict()

for name, model in models.items():
    measurements[name] = list()
    for s in range(5, 150, 5):
        times = list()
        for _ in range(10 if len(model.metabolites) < 1000 else 1):
            if len(model.metabolites) >= s:
                times.append(sample_analysis_time(model.copy(), s))
        measurements[name].append(np.mean(times))
        clear_output()
        print(measurements)

{'RECON1': [418.72530460357666, 412.527259349823, 422.71280956268311]}


In [ ]:
df = pd.DataFrame(measurements)

In [ ]:
df.plot()

In [ ]:
df